In [1]:
from glob import glob

ass_paths = glob("../data/Subtitles/*.ass")
srt_paths = glob("../data/Subtitles/*.srt")

subtitles_paths = sorted(ass_paths + srt_paths)

subtitles_paths[:5]

['../data/Subtitles\\Naruto 001 English.srt',
 '../data/Subtitles\\Naruto 002 English.srt',
 '../data/Subtitles\\Naruto 003 English.srt',
 '../data/Subtitles\\Naruto 004 English.srt',
 '../data/Subtitles\\Naruto 005 English.srt']

In [2]:
import re

scripts = []
episode_num = []

for path in subtitles_paths:
    with open(path, "r", encoding="utf-8", errors="ignore") as file:
        lines = file.readlines()

        # Skip header lines if it's an .ass file
        if path.endswith(".ass"):
            lines = lines[27:]
            rows = [",".join(line.split(",")[9:]) for line in lines]
            rows = [line.replace("\\N", " ") for line in rows]
            script = " ".join(rows)

        elif path.endswith(".srt"):
            # Basic .srt parsing: skip subtitle numbers and timestamps
            lines = [line.strip() for line in lines if line.strip()]
            rows = [
                line
                for line in lines
                if not re.match(r"^\d+$", line)
                and not re.match(r"^\d{2}:\d{2}:\d{2},\d{3}", line)
            ]
            script = " ".join(rows)

        # Try to extract episode number using regex
        filename = path.split("/")[-1]
        match = re.search(r"\d+", filename)
        if match:
            episode = int(match.group())
            scripts.append(script)
            episode_num.append(episode)
        else:
            print(f"⚠️ Couldn't find episode number in filename: {filename}")

In [3]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

df = pd.DataFrame.from_dict({"episode": episode_num, "script": scripts})

In [5]:
df.head()

,episode,script
0,1,Much time makes existed a demon vixen with nin...
1,2,! ! From it releases! It goes well. But after ...
2,3,Guideline 9 AM Today it is a special day Itada...
3,4,"E' in delay! Naruto, you are a po' firm! Becau..."
4,5,You must use of the techniques using the head ...


In [6]:
%pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [11]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/12.8 MB 3.2 MB/s eta 0:00:04
     ----- ---------------------------------- 1.8/12.8 MB 3.4 MB/s eta 0:00:04
     -------- ------------------------------- 2.6/12.8 MB 3.6 MB/s eta 0:00:03
     ----------- ---------------------------- 3.7/12.8 MB 3.9 MB/s eta 0:00:03
     -------------- ------------------------- 4.7/12.8 MB 4.0 MB/s eta 0:00:03
     ------------------ --------------------- 5.8/12.8 MB 4.2 MB/s eta 0:00:02
     --------------------- ------------------ 6.8/12.8 MB 4.4 MB/s eta 0:00:02
     ------------------------- -------------- 8.1/12.8 MB 4.6 MB/s eta 0:00:02
     ----------------------------- ---------- 9.4/12.8 MB 4.7 MB/s eta 0:00:01
     -------------------------------- ------- 10.5/12.8 MB 4.8 MB/s eta 0:00:01
     ------------------------------------ --- 11.8/12.8 MB 4.9 MB

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [14]:
doc = nlp("Mark went to Germany")

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Germany GPE


In [17]:
%pip install nltk

  Using cached regex-2025.7.34-cp313-cp313-win_amd64.whl.metadata (41 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 1.5 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 1.6 MB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 1.7 MB/s  0:00:00
Using cached regex-2025.7.34-cp313-cp313-win_amd64.whl (275 kB)

   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   -------------------------- ------------- 2/3 [nltk]
   ----------------------

In [ ]:
from nltk import sent_tokenize


def get_ners(script):
    script_sentences = sent_tokenize(script)

    ner_output = []

    for sentence in script_sentences:
        doc = nlp(sentence)
        ners = set()
        for ent in docs.ents:
            if ent.label_ == "PERSON":
                full_name = ent.text
                first_name = full_name.split(" ")[0]
                ners.add(first_name)
        ner_output.append(list(ners))
    return ner_output

In [ ]:
df["ners"] = df["script"].apply(get_ners)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\arnob/nltk_data'
    - 'c:\\Users\\arnob\\OneDrive\\Desktop\\Projects\\shinobinet\\.venv\\nltk_data'
    - 'c:\\Users\\arnob\\OneDrive\\Desktop\\Projects\\shinobinet\\.venv\\share\\nltk_data'
    - 'c:\\Users\\arnob\\OneDrive\\Desktop\\Projects\\shinobinet\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\arnob\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [25]:
window = 10
entity_relationship = []

for row in df["ners"]:
    previous_entitties_in_window = []

    for sentence in row:
        previous_entitties_in_window.append(sentence)
        previous_entities_in_window = previous_entities_in_window[-10:]

        previous_entities_flattened = sum(previous_entities_in_window, [])

        for entity in sentence:
            for entity_in_window in previous_entities_flattened:
                if entity != entity_in_window:
                    entity_rel = sorted([entity, entity_in_window])
                    entity_relationship.append(entity_rel)

KeyError: 'ners'

In [24]:
relationship_df = pd.DataFrame({"value": entity_relationship})

NameError: name 'entity_relationship' is not defined

In [ ]:
relationship_df["source"] = relationship_df["value"].apply(lambda x: x[0])
relationship_df["target"] = relationship_df["value"].apply(lambda x: x[1])

In [ ]:
relationship_df = relationship_df.groupby(["source", "target"]).count().reset_index()

In [ ]:
relationship_df = relationship_df.sort_values("value", ascending=False)
relationship_df.head()

In [ ]:
relationship_df = relationship_df.head(200)

In [ ]:
import networkx as nx

G = nx.from_pandas_edgelist(
    relationship_df,
    source="source",
    target="target",
    edge_attr="value",
    create_using=nx.Graph(),
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
pos = nx.kamada_kawai_layout(G)
nx.draw(G, with_labels=True, node_color="skyblue", edge_cmap=plt.cm.Blues, pos=pos)
plt.show()

In [26]:
from pyvis.network import Network

net = Network(
    notebook=True, width="1000px", height="700px", bgcolor="#222222", font_color="white"
)

node_degree = dict(G.degree)

nx.set_node_attributes(G, node_degree, "size")

net.from_nx(G)
net.show("naruto.html")

ModuleNotFoundError: No module named 'pyvis'